In [12]:
from model.LTA_Stacked import LTA_Stacked
from model import load_model
import torch
from hvo_sequence.hvo_seq import HVO_Sequence
from hvo_sequence.drum_mappings import ROLAND_REDUCED_MAPPING
from bokeh.plotting import show, output_notebook
import os
import timeit
from hvo_sequence.hvo_seq import HVO_Sequence

output_notebook()





import numpy as np
import torch
import tqdm


def get_data(model_folder = '[Larger srcMasked 30prcnt] [bassGuiSynLead] no in vel_itzc90b6',
            model_name = '080.pth',
            use_bernoulli = True,
            threshold = 0.5,
            groove_threshold = 0.6,
            prime_bars = 0,
            predict_n_steps_at_a_time = 16):
    full_path = f'{model_folder}/{model_name}/bern_{use_bernoulli}_prime_{prime_bars}_bars_thres_{threshold:.2f}_gthresh_{groove_threshold:.2f}_n_steps_per_gen_{predict_n_steps_at_a_time}'
    generated_drums = torch.tensor(np.load(f'{full_path}/generated_drums.npy'))
    generated_groove = torch.tensor(np.load(f'{full_path}/generated_grooves.npy'))
    gt_groove = torch.tensor(np.load(f'{full_path}/groove_hvos_gt.npy'))
    gt_drums = torch.tensor(np.load(f'{full_path}/drum_hvos_gt.npy'))
    return generated_drums, generated_groove, gt_groove, gt_drums


Loading BokehJS ...

In [13]:
# UTils
from eval.LTA import jaccard_similarity, velocity_distribution, offset_distribution, get_as_hvo_sequence, get_subsets_hit_vel_stats, get_overlapping_ration, extract_SSM_data


In [14]:
# Generated Drum Stats
print('------------------------------------')
print(f'---- Primed Bars: 0 ----')
print('Table 6 in Paper - D2D Analysis')
print('Drum2Drum Analysis - With Instrumental Groove')
print('Generated Drums Compared Against GT Drums')
print('------------------------------------')

generated_drums, generated_groove, gt_groove, gt_drums = get_data(prime_bars = 0)

hvo_seqs_generated_drums = []
hvo_seqs_drums_gt = []

flattened_versions_drums_generated = []
flattened_versions_drums_gt = []
full_versions_generated_drums = []
full_versions_drums_gt = []

hit_sims_full_generated_drum2gt_drum = []
hit_sims_full_gFromdrumGen2gFromDrumgt = []

vel_distr_drum = []
vel_distr_drum_gt = []

vel_distr_groove_from_drum_generated = []
vel_distr_groove_from_drum_gt = []

offset_distr_drum = []
offset_distr_drum_gt = []

offset_distr_groove_from_drum_generated = []
offset_distr_groove_from_drum_gt = []

for k in tqdm.trange(generated_drums.shape[0]):
            full_versions_generated_drums.append(generated_drums[k])
            full_versions_drums_gt.append(gt_drums[k])
    
            hvsq_gt = get_as_hvo_sequence(gt_drums[k])
            hvsq_gen = get_as_hvo_sequence(generated_drums[k])
    
            hvo_seqs_generated_drums.append(hvsq_gen)
            hvo_seqs_drums_gt.append(hvsq_gt)
    
            flattened_versions_drums_generated.append(torch.tensor(hvsq_gen.flatten_voices(reduce_dim=True)))
            flattened_versions_drums_gt.append(torch.tensor(hvsq_gt.flatten_voices(reduce_dim=True)))
    
            hit_sims_full_generated_drum2gt_drum.append(jaccard_similarity(generated_drums[k], gt_drums[k], has_batch_dim=False).item())
            hit_sims_full_gFromdrumGen2gFromDrumgt.append(jaccard_similarity(flattened_versions_drums_generated[-1], flattened_versions_drums_gt[-1], has_batch_dim=False).item())
    
            vel_distr_drum.append(velocity_distribution(generated_drums, only_at_hits=True))
            vel_distr_drum_gt.append(velocity_distribution(gt_drums, only_at_hits=True))
    
            vel_distr_groove_from_drum_generated.append(velocity_distribution(flattened_versions_drums_generated[-1].unsqueeze(0), only_at_hits=True))
            vel_distr_groove_from_drum_gt.append(velocity_distribution(flattened_versions_drums_gt[-1].unsqueeze(0), only_at_hits=True))
    
    
            offset_distr_drum.append(offset_distribution(generated_drums, only_at_hits=True))
            offset_distr_drum_gt.append(offset_distribution(gt_drums, only_at_hits=True))
    
            offset_distr_groove_from_drum_generated.append(offset_distribution(flattened_versions_drums_generated[-1].unsqueeze(0), only_at_hits=True))
            offset_distr_groove_from_drum_gt.append(offset_distribution(flattened_versions_drums_gt[-1].unsqueeze(0), only_at_hits=True))
            
        
print('Drum2Drum - With Instrumental Groove')
print(f'hit_sims_full_generated_drum2gt_drum {np.round(np.nanmean(hit_sims_full_generated_drum2gt_drum), 2)} ± {np.round(np.nanstd(hit_sims_full_generated_drum2gt_drum), 2)}')
print(f'vel_distr_drum {np.round(np.nanmean([x[0] for x in vel_distr_drum]), 2)} ± {np.round(np.nanmean([x[1] for x in vel_distr_drum]), 2)}')
print(f'vel_distr_drum_gt {np.round(np.nanmean([x[0] for x in vel_distr_drum_gt]), 2)} ± {np.round(np.nanmean([x[1] for x in vel_distr_drum_gt]), 2)}')
print(f'offset_distr_drum {np.round(np.nanmean([x[0] for x in offset_distr_drum]), 2)} ± {np.round(np.nanmean([x[1] for x in offset_distr_drum]), 2)}')
print(f'offset_distr_drum_gt {np.round(np.nanmean([x[0] for x in offset_distr_drum_gt]), 2)} ± {np.round(np.nanmean([x[1] for x in offset_distr_drum_gt]), 2)}')

print('DrumGeneratedGroove2DrumGTGroove - With Instrumental Groove')
print(f'hit_sims_full_gFromdrumGen2gFromDrumgt {np.round(np.nanmean(hit_sims_full_gFromdrumGen2gFromDrumgt), 2)} ± {np.round(np.nanstd(hit_sims_full_gFromdrumGen2gFromDrumgt), 2)}')
print(f'vel_distr_groove_from_drum_generated {np.round(np.nanmean([x[0] for x in vel_distr_groove_from_drum_generated]), 2)} ± {np.round(np.nanmean([x[1] for x in vel_distr_groove_from_drum_generated]), 2)}')
print(f'vel_distr_groove_from_drum_gt {np.round(np.nanmean([x[0] for x in vel_distr_groove_from_drum_gt]), 2)} ± {np.round(np.nanmean([x[1] for x in vel_distr_groove_from_drum_gt]), 2)}')
print(f'offset_distr_groove_from_drum_generated {np.round(np.nanmean([x[0] for x in offset_distr_groove_from_drum_generated]), 2)} ± {np.round(np.nanmean([x[1] for x in offset_distr_groove_from_drum_generated]), 2)}')
print(f'offset_distr_groove_from_drum_gt {np.round(np.nanmean([x[0] for x in offset_distr_groove_from_drum_gt]), 2)} ± {np.round(np.nanmean([x[1] for x in offset_distr_groove_from_drum_gt]), 2)}')

# Generated Drum Stats
print('------------------------------------')
print(f'---- Primed Bars: 2 ----')
print('Table 6 in Paper - Polyphonic Analysis')
print('Drum2Drum Analysis - With Instrumental Groove')
print('Generated Drums Compared Against GT Drums')
print('------------------------------------')

generated_drums, generated_groove, gt_groove, gt_drums = get_data(prime_bars = 2)

hvo_seqs_generated_drums = []
hvo_seqs_drums_gt = []

flattened_versions_drums_generated = []
flattened_versions_drums_gt = []
full_versions_generated_drums = []
full_versions_drums_gt = []

hit_sims_full_generated_drum2gt_drum = []
hit_sims_full_gFromdrumGen2gFromDrumgt = []

vel_distr_drum = []
vel_distr_drum_gt = []

vel_distr_groove_from_drum_generated = []
vel_distr_groove_from_drum_gt = []

offset_distr_drum = []
offset_distr_drum_gt = []

offset_distr_groove_from_drum_generated = []
offset_distr_groove_from_drum_gt = []

for k in tqdm.trange(generated_drums.shape[0]):
            full_versions_generated_drums.append(generated_drums[k])
            full_versions_drums_gt.append(gt_drums[k])
    
            hvsq_gt = get_as_hvo_sequence(gt_drums[k])
            hvsq_gen = get_as_hvo_sequence(generated_drums[k])
    
            hvo_seqs_generated_drums.append(hvsq_gen)
            hvo_seqs_drums_gt.append(hvsq_gt)
    
            flattened_versions_drums_generated.append(torch.tensor(hvsq_gen.flatten_voices(reduce_dim=True)))
            flattened_versions_drums_gt.append(torch.tensor(hvsq_gt.flatten_voices(reduce_dim=True)))
    
            hit_sims_full_generated_drum2gt_drum.append(jaccard_similarity(generated_drums[k], gt_drums[k], has_batch_dim=False).item())
            hit_sims_full_gFromdrumGen2gFromDrumgt.append(jaccard_similarity(flattened_versions_drums_generated[-1], flattened_versions_drums_gt[-1], has_batch_dim=False).item())
    
            vel_distr_drum.append(velocity_distribution(generated_drums, only_at_hits=True))
            vel_distr_drum_gt.append(velocity_distribution(gt_drums, only_at_hits=True))
    
            vel_distr_groove_from_drum_generated.append(velocity_distribution(flattened_versions_drums_generated[-1].unsqueeze(0), only_at_hits=True))
            vel_distr_groove_from_drum_gt.append(velocity_distribution(flattened_versions_drums_gt[-1].unsqueeze(0), only_at_hits=True))
    
    
            offset_distr_drum.append(offset_distribution(generated_drums, only_at_hits=True))
            offset_distr_drum_gt.append(offset_distribution(gt_drums, only_at_hits=True))
    
            offset_distr_groove_from_drum_generated.append(offset_distribution(flattened_versions_drums_generated[-1].unsqueeze(0), only_at_hits=True))
            offset_distr_groove_from_drum_gt.append(offset_distribution(flattened_versions_drums_gt[-1].unsqueeze(0), only_at_hits=True))
            
        
print('Drum2Drum - With Instrumental Groove')
print(f'hit_sims_full_generated_drum2gt_drum {np.round(np.nanmean(hit_sims_full_generated_drum2gt_drum), 2)} ± {np.round(np.nanstd(hit_sims_full_generated_drum2gt_drum), 2)}')
print(f'vel_distr_drum {np.round(np.nanmean([x[0] for x in vel_distr_drum]), 2)} ± {np.round(np.nanmean([x[1] for x in vel_distr_drum]), 2)}')
print(f'vel_distr_drum_gt {np.round(np.nanmean([x[0] for x in vel_distr_drum_gt]), 2)} ± {np.round(np.nanmean([x[1] for x in vel_distr_drum_gt]), 2)}')
print(f'offset_distr_drum {np.round(np.nanmean([x[0] for x in offset_distr_drum]), 2)} ± {np.round(np.nanmean([x[1] for x in offset_distr_drum]), 2)}')
print(f'offset_distr_drum_gt {np.round(np.nanmean([x[0] for x in offset_distr_drum_gt]), 2)} ± {np.round(np.nanmean([x[1] for x in offset_distr_drum_gt]), 2)}')

print('DrumGeneratedGroove2DrumGTGroove - With Instrumental Groove')
print(f'hit_sims_full_gFromdrumGen2gFromDrumgt {np.round(np.nanmean(hit_sims_full_gFromdrumGen2gFromDrumgt), 2)} ± {np.round(np.nanstd(hit_sims_full_gFromdrumGen2gFromDrumgt), 2)}')
print(f'vel_distr_groove_from_drum_generated {np.round(np.nanmean([x[0] for x in vel_distr_groove_from_drum_generated]), 2)} ± {np.round(np.nanmean([x[1] for x in vel_distr_groove_from_drum_generated]), 2)}')
print(f'vel_distr_groove_from_drum_gt {np.round(np.nanmean([x[0] for x in vel_distr_groove_from_drum_gt]), 2)} ± {np.round(np.nanmean([x[1] for x in vel_distr_groove_from_drum_gt]), 2)}')
print(f'offset_distr_groove_from_drum_generated {np.round(np.nanmean([x[0] for x in offset_distr_groove_from_drum_generated]), 2)} ± {np.round(np.nanmean([x[1] for x in offset_distr_groove_from_drum_generated]), 2)}')
print(f'offset_distr_groove_from_drum_gt {np.round(np.nanmean([x[0] for x in offset_distr_groove_from_drum_gt]), 2)} ± {np.round(np.nanmean([x[1] for x in offset_distr_groove_from_drum_gt]), 2)}')

# # Generated Drum Stats
# print('------------------------------------')
# print(f'---- Primed Bars: 4 ----')
# print('Table 6 in Paper - Polyphonic Analysis')
# print('Drum2Drum Analysis - With Instrumental Groove')
# print('Generated Drums Compared Against GT Drums')
# print('------------------------------------')
# 
# generated_drums, generated_groove, gt_groove, gt_drums = get_data(prime_bars = 4)
# 
# hvo_seqs_generated_drums = []
# hvo_seqs_drums_gt = []
# 
# flattened_versions_drums_generated = []
# flattened_versions_drums_gt = []
# full_versions_generated_drums = []
# full_versions_drums_gt = []
# 
# hit_sims_full_generated_drum2gt_drum = []
# hit_sims_full_gFromdrumGen2gFromDrumgt = []
# 
# vel_distr_drum = []
# vel_distr_drum_gt = []
# 
# vel_distr_groove_from_drum_generated = []
# vel_distr_groove_from_drum_gt = []
# 
# offset_distr_drum = []
# offset_distr_drum_gt = []
# 
# offset_distr_groove_from_drum_generated = []
# offset_distr_groove_from_drum_gt = []
# 
# for k in tqdm.trange(generated_drums.shape[0]):
#             full_versions_generated_drums.append(generated_drums[k])
#             full_versions_drums_gt.append(gt_drums[k])
#     
#             hvsq_gt = get_as_hvo_sequence(gt_drums[k])
#             hvsq_gen = get_as_hvo_sequence(generated_drums[k])
#     
#             hvo_seqs_generated_drums.append(hvsq_gen)
#             hvo_seqs_drums_gt.append(hvsq_gt)
#     
#             flattened_versions_drums_generated.append(torch.tensor(hvsq_gen.flatten_voices(reduce_dim=True)))
#             flattened_versions_drums_gt.append(torch.tensor(hvsq_gt.flatten_voices(reduce_dim=True)))
#     
#             hit_sims_full_generated_drum2gt_drum.append(jaccard_similarity(generated_drums[k], gt_drums[k], has_batch_dim=False).item())
#             hit_sims_full_gFromdrumGen2gFromDrumgt.append(jaccard_similarity(flattened_versions_drums_generated[-1], flattened_versions_drums_gt[-1], has_batch_dim=False).item())
#     
#             vel_distr_drum.append(velocity_distribution(generated_drums, only_at_hits=True))
#             vel_distr_drum_gt.append(velocity_distribution(gt_drums, only_at_hits=True))
#     
#             vel_distr_groove_from_drum_generated.append(velocity_distribution(flattened_versions_drums_generated[-1].unsqueeze(0), only_at_hits=True))
#             vel_distr_groove_from_drum_gt.append(velocity_distribution(flattened_versions_drums_gt[-1].unsqueeze(0), only_at_hits=True))
#     
#     
#             offset_distr_drum.append(offset_distribution(generated_drums, only_at_hits=True))
#             offset_distr_drum_gt.append(offset_distribution(gt_drums, only_at_hits=True))
#     
#             offset_distr_groove_from_drum_generated.append(offset_distribution(flattened_versions_drums_generated[-1].unsqueeze(0), only_at_hits=True))
#             offset_distr_groove_from_drum_gt.append(offset_distribution(flattened_versions_drums_gt[-1].unsqueeze(0), only_at_hits=True))
#             
#         
# print('Drum2Drum - With Instrumental Groove')
# print(f'hit_sims_full_generated_drum2gt_drum {np.round(np.nanmean(hit_sims_full_generated_drum2gt_drum), 2)} ± {np.round(np.nanstd(hit_sims_full_generated_drum2gt_drum), 2)}')
# print(f'vel_distr_drum {np.round(np.nanmean([x[0] for x in vel_distr_drum]), 2)} ± {np.round(np.nanmean([x[1] for x in vel_distr_drum]), 2)}')
# print(f'vel_distr_drum_gt {np.round(np.nanmean([x[0] for x in vel_distr_drum_gt]), 2)} ± {np.round(np.nanmean([x[1] for x in vel_distr_drum_gt]), 2)}')
# print(f'offset_distr_drum {np.round(np.nanmean([x[0] for x in offset_distr_drum]), 2)} ± {np.round(np.nanmean([x[1] for x in offset_distr_drum]), 2)}')
# print(f'offset_distr_drum_gt {np.round(np.nanmean([x[0] for x in offset_distr_drum_gt]), 2)} ± {np.round(np.nanmean([x[1] for x in offset_distr_drum_gt]), 2)}')
# 
# print('DrumGeneratedGroove2DrumGTGroove - With Instrumental Groove')
# print(f'hit_sims_full_gFromdrumGen2gFromDrumgt {np.round(np.nanmean(hit_sims_full_gFromdrumGen2gFromDrumgt), 2)} ± {np.round(np.nanstd(hit_sims_full_gFromdrumGen2gFromDrumgt), 2)}')
# print(f'vel_distr_groove_from_drum_generated {np.round(np.nanmean([x[0] for x in vel_distr_groove_from_drum_generated]), 2)} ± {np.round(np.nanmean([x[1] for x in vel_distr_groove_from_drum_generated]), 2)}')
# print(f'vel_distr_groove_from_drum_gt {np.round(np.nanmean([x[0] for x in vel_distr_groove_from_drum_gt]), 2)} ± {np.round(np.nanmean([x[1] for x in vel_distr_groove_from_drum_gt]), 2)}')
# print(f'offset_distr_groove_from_drum_generated {np.round(np.nanmean([x[0] for x in offset_distr_groove_from_drum_generated]), 2)} ± {np.round(np.nanmean([x[1] for x in offset_distr_groove_from_drum_generated]), 2)}')
# print(f'offset_distr_groove_from_drum_gt {np.round(np.nanmean([x[0] for x in offset_distr_groove_from_drum_gt]), 2)} ± {np.round(np.nanmean([x[1] for x in offset_distr_groove_from_drum_gt]), 2)}')

------------------------------------
---- Primed Bars: 0 ----
Table 6 in Paper - D2D Analysis
Drum2Drum Analysis - With Instrumental Groove
Generated Drums Compared Against GT Drums
------------------------------------


100%|██████████| 3014/3014 [02:39<00:00, 18.93it/s]


Drum2Drum - With Instrumental Groove
hit_sims_full_generated_drum2gt_drum 0.22 ± 0.15
vel_distr_drum 0.57 ± 0.2
vel_distr_drum_gt 0.68 ± 0.21
offset_distr_drum -0.01 ± 0.05
offset_distr_drum_gt 0.0 ± 0.12
DrumGeneratedGroove2DrumGTGroove - With Instrumental Groove
hit_sims_full_gFromdrumGen2gFromDrumgt 0.48 ± 0.21
vel_distr_groove_from_drum_generated 0.58 ± 0.17
vel_distr_groove_from_drum_gt 0.71 ± 0.16
offset_distr_groove_from_drum_generated -0.01 ± 0.03
offset_distr_groove_from_drum_gt -0.01 ± 0.07
------------------------------------
---- Primed Bars: 2 ----
Table 6 in Paper - Polyphonic Analysis
Drum2Drum Analysis - With Instrumental Groove
Generated Drums Compared Against GT Drums
------------------------------------


100%|██████████| 3014/3014 [02:32<00:00, 19.76it/s]

Drum2Drum - With Instrumental Groove
hit_sims_full_generated_drum2gt_drum 0.45 ± 0.23
vel_distr_drum 0.61 ± 0.2
vel_distr_drum_gt 0.68 ± 0.21
offset_distr_drum -0.01 ± 0.05
offset_distr_drum_gt 0.0 ± 0.12
DrumGeneratedGroove2DrumGTGroove - With Instrumental Groove
hit_sims_full_gFromdrumGen2gFromDrumgt 0.69 ± 0.2
vel_distr_groove_from_drum_generated 0.63 ± 0.18
vel_distr_groove_from_drum_gt 0.71 ± 0.16
offset_distr_groove_from_drum_generated -0.01 ± 0.03
offset_distr_groove_from_drum_gt -0.01 ± 0.07


In [15]:

# Generated Groove Stats
print('------------------------------------')
print(f'---- Primed Bars: 0 ----')
print('Table 6 in Paper - G2G Analysis')
print('Groove2Groove Analysis - With Instrumental Groove')
print('Generated Grooves Compared Against GT Grooves')
print('------------------------------------')

generated_drums, generated_groove, gt_groove, gt_drums = get_data(prime_bars = 0)


full_versions_generated_grooves = []
full_versions_grooves_gt = []

hit_sims_full_generated_groove2gt_groove = []
hit_sims_full_gFromgrooveGen2gFromgroovegt = []

vel_distr_groove = []
vel_distr_groove_gt = []

vel_distr_groove_from_groove_generated = []
vel_distr_groove_from_groove_gt = []

offset_distr_groove = []
offset_distr_groove_gt = []

offset_distr_groove_from_groove_generated = []
offset_distr_groove_from_groove_gt = []

style_consistency = []

for k in range(generated_groove.shape[0]):
    full_versions_generated_grooves.append(generated_groove[k])
    full_versions_grooves_gt.append(gt_groove[k])


    hit_sims_full_generated_groove2gt_groove.append(jaccard_similarity(generated_groove[k], gt_groove[k], has_batch_dim=False).item())

    vel_distr_groove.append(velocity_distribution(generated_groove[k:k+1], only_at_hits=True))
    vel_distr_groove_gt.append(velocity_distribution(gt_groove[k:k+1], only_at_hits=True))



    offset_distr_groove.append(offset_distribution(generated_groove[k:k+1], only_at_hits=True))
    offset_distr_groove_gt.append(offset_distribution(gt_groove[k:k+1], only_at_hits=True))
    
print('groove2groove - With Instrumental Groove')
print(f'hit_sims_full_generated_groove2gt_groove {np.round(np.nanmean(hit_sims_full_generated_groove2gt_groove), 2)} ± {np.round(np.nanstd(hit_sims_full_generated_groove2gt_groove), 2)}')
print(f'vel_distr_groove {np.round(np.nanmean([x[0] for x in vel_distr_groove]), 2)} ± {np.round(np.nanmean([x[1] for x in vel_distr_groove]), 2)}')
print(f'vel_distr_groove_gt {np.round(np.nanmean([x[0] for x in vel_distr_groove_gt]), 2)} ± {np.round(np.nanmean([x[1] for x in vel_distr_groove_gt]), 2)}')
print(f'offset_distr_groove {np.round(np.nanmean([x[0] for x in offset_distr_groove]), 2)} ± {np.round(np.nanmean([x[1] for x in offset_distr_groove]), 2)}')
print(f'offset_distr_groove_gt {np.round(np.nanmean([x[0] for x in offset_distr_groove_gt]), 2)} ± {np.round(np.nanmean([x[1] for x in offset_distr_groove_gt]), 2)}')





# Generated Groove Stats
print('------------------------------------')
print(f'---- Primed Bars: 2 ----')
print('Table 7 in Paper - Polyphonic Analysis')
print('Groove2Groove Analysis - With Instrumental Groove')
print('Generated Grooves Compared Against GT Grooves')
print('------------------------------------')

generated_drums, generated_groove, gt_groove, gt_drums = get_data(prime_bars = 2)


full_versions_generated_grooves = []
full_versions_grooves_gt = []

hit_sims_full_generated_groove2gt_groove = []
hit_sims_full_gFromgrooveGen2gFromgroovegt = []

vel_distr_groove = []
vel_distr_groove_gt = []

vel_distr_groove_from_groove_generated = []
vel_distr_groove_from_groove_gt = []

offset_distr_groove = []
offset_distr_groove_gt = []

offset_distr_groove_from_groove_generated = []
offset_distr_groove_from_groove_gt = []

style_consistency = []

for k in range(generated_groove.shape[0]):
    full_versions_generated_grooves.append(generated_groove[k])
    full_versions_grooves_gt.append(gt_groove[k])


    hit_sims_full_generated_groove2gt_groove.append(jaccard_similarity(generated_groove[k], gt_groove[k], has_batch_dim=False).item())

    vel_distr_groove.append(velocity_distribution(generated_groove[k:k+1], only_at_hits=True))
    vel_distr_groove_gt.append(velocity_distribution(gt_groove[k:k+1], only_at_hits=True))



    offset_distr_groove.append(offset_distribution(generated_groove[k:k+1], only_at_hits=True))
    offset_distr_groove_gt.append(offset_distribution(gt_groove[k:k+1], only_at_hits=True))
    
print('groove2groove - With Instrumental Groove')
print(f'hit_sims_full_generated_groove2gt_groove {np.round(np.nanmean(hit_sims_full_generated_groove2gt_groove), 2)} ± {np.round(np.nanstd(hit_sims_full_generated_groove2gt_groove), 2)}')
print(f'vel_distr_groove {np.round(np.nanmean([x[0] for x in vel_distr_groove]), 2)} ± {np.round(np.nanmean([x[1] for x in vel_distr_groove]), 2)}')
print(f'vel_distr_groove_gt {np.round(np.nanmean([x[0] for x in vel_distr_groove_gt]), 2)} ± {np.round(np.nanmean([x[1] for x in vel_distr_groove_gt]), 2)}')
print(f'offset_distr_groove {np.round(np.nanmean([x[0] for x in offset_distr_groove]), 2)} ± {np.round(np.nanmean([x[1] for x in offset_distr_groove]), 2)}')
print(f'offset_distr_groove_gt {np.round(np.nanmean([x[0] for x in offset_distr_groove_gt]), 2)} ± {np.round(np.nanmean([x[1] for x in offset_distr_groove_gt]), 2)}')


# # Generated Groove Stats
# print('------------------------------------')
# print(f'---- Primed Bars: 4 ----')
# print('Table 7 in Paper - Polyphonic Analysis')
# print('Groove2Groove Analysis - With Instrumental Groove')
# print('Generated Grooves Compared Against GT Grooves')
# print('------------------------------------')
# 
# generated_drums, generated_groove, gt_groove, gt_drums = get_data(prime_bars = 4)
# 
# 
# full_versions_generated_grooves = []
# full_versions_grooves_gt = []
# 
# hit_sims_full_generated_groove2gt_groove = []
# hit_sims_full_gFromgrooveGen2gFromgroovegt = []
# 
# vel_distr_groove = []
# vel_distr_groove_gt = []
# 
# vel_distr_groove_from_groove_generated = []
# vel_distr_groove_from_groove_gt = []
# 
# offset_distr_groove = []
# offset_distr_groove_gt = []
# 
# offset_distr_groove_from_groove_generated = []
# offset_distr_groove_from_groove_gt = []
# 
# style_consistency = []
# 
# for k in range(generated_groove.shape[0]):
#     full_versions_generated_grooves.append(generated_groove[k])
#     full_versions_grooves_gt.append(gt_groove[k])
# 
# 
#     hit_sims_full_generated_groove2gt_groove.append(jaccard_similarity(generated_groove[k], gt_groove[k], has_batch_dim=False).item())
# 
#     vel_distr_groove.append(velocity_distribution(generated_groove[k:k+1], only_at_hits=True))
#     vel_distr_groove_gt.append(velocity_distribution(gt_groove[k:k+1], only_at_hits=True))
# 
# 
# 
#     offset_distr_groove.append(offset_distribution(generated_groove[k:k+1], only_at_hits=True))
#     offset_distr_groove_gt.append(offset_distribution(gt_groove[k:k+1], only_at_hits=True))
#     
# print('groove2groove - With Instrumental Groove')
# print(f'hit_sims_full_generated_groove2gt_groove {np.round(np.nanmean(hit_sims_full_generated_groove2gt_groove), 2)} ± {np.round(np.nanstd(hit_sims_full_generated_groove2gt_groove), 2)}')
# print(f'vel_distr_groove {np.round(np.nanmean([x[0] for x in vel_distr_groove]), 2)} ± {np.round(np.nanmean([x[1] for x in vel_distr_groove]), 2)}')
# print(f'vel_distr_groove_gt {np.round(np.nanmean([x[0] for x in vel_distr_groove_gt]), 2)} ± {np.round(np.nanmean([x[1] for x in vel_distr_groove_gt]), 2)}')
# print(f'offset_distr_groove {np.round(np.nanmean([x[0] for x in offset_distr_groove]), 2)} ± {np.round(np.nanmean([x[1] for x in offset_distr_groove]), 2)}')
# print(f'offset_distr_groove_gt {np.round(np.nanmean([x[0] for x in offset_distr_groove_gt]), 2)} ± {np.round(np.nanmean([x[1] for x in offset_distr_groove_gt]), 2)}')


------------------------------------
---- Primed Bars: 0 ----
Table 6 in Paper - G2G Analysis
Groove2Groove Analysis - With Instrumental Groove
Generated Grooves Compared Against GT Grooves
------------------------------------
groove2groove - With Instrumental Groove
hit_sims_full_generated_groove2gt_groove 0.24 ± 0.17
vel_distr_groove 0.64 ± 0.1
vel_distr_groove_gt 0.75 ± 0.1
offset_distr_groove -0.03 ± 0.03
offset_distr_groove_gt -0.02 ± 0.1
------------------------------------
---- Primed Bars: 2 ----
Table 7 in Paper - Polyphonic Analysis
Groove2Groove Analysis - With Instrumental Groove
Generated Grooves Compared Against GT Grooves
------------------------------------
groove2groove - With Instrumental Groove
hit_sims_full_generated_groove2gt_groove 0.49 ± 0.25
vel_distr_groove 0.67 ± 0.1
vel_distr_groove_gt 0.75 ± 0.1
offset_distr_groove -0.03 ± 0.04
offset_distr_groove_gt -0.02 ± 0.1


In [17]:
# Overlap Hit Ratio 
print('------------------------------------')
print(f'---- Primed Bars: 0 ----')
print('Table 8 in Paper - Overlap Hit Ratio')
print('------------------------------------')

generated_drums, generated_groove, gt_groove, gt_drums = get_data(prime_bars = 0)

print("Drums Generation")
print(get_subsets_hit_vel_stats(generated_drums))

print("Groove Generation")
print(get_subsets_hit_vel_stats(generated_groove))

print("Percent Overlapping Generated Groove")
print(get_overlapping_ration(generated_drums, generated_groove))

print("Percent Overlapping GT Groove")
print(get_overlapping_ration(generated_drums, gt_groove))

# Overlap Hit Ratio 
print('------------------------------------')
print(f'---- Primed Bars: 2 ----')
print('Table 8 in Paper - Overlap Hit Ratio')
print('------------------------------------')

generated_drums, generated_groove, gt_groove, gt_drums = get_data(prime_bars = 2)

print("Drums Generation")
print(get_subsets_hit_vel_stats(generated_drums))

print("Groove Generation")
print(get_subsets_hit_vel_stats(generated_groove))

print("Percent Overlapping Generated Groove")
print(get_overlapping_ration(generated_drums, generated_groove))

print("Percent Overlapping GT Groove")
print(get_overlapping_ration(generated_drums, gt_groove))

# # Overlap Hit Ratio 
# print('------------------------------------')
# print(f'---- Primed Bars: 4 ----')
# print('Table 8 in Paper - Overlap Hit Ratio')
# print('------------------------------------')
# 
# generated_drums, generated_groove, gt_groove, gt_drums = get_data(prime_bars = 4)
# 
# print("Drums Generation")
# print(get_subsets_hit_vel_stats(generated_drums))
# 
# print("Groove Generation")
# print(get_subsets_hit_vel_stats(generated_groove))
# 
# print("Percent Overlapping Generated Groove")
# print(get_overlapping_ration(generated_drums, generated_groove))
# 
# print("Percent Overlapping GT Groove")
# print(get_overlapping_ration(generated_drums, gt_groove))



------------------------------------
---- Primed Bars: 0 ----
Table 8 in Paper - Overlap Hit Ratio
------------------------------------
Drums Generation
9
('0.57 ± 0.20', '-0.01 ± 0.05', '255.37 ± 155.63')
Groove Generation
1
('0.65 ± 0.13', '-0.05 ± 0.08', '72.98 ± 63.16')
Percent Overlapping Generated Groove
22.77 ± 18.19 %
Percent Overlapping GT Groove
47.95 ± 19.42 %
------------------------------------
---- Primed Bars: 2 ----
Table 8 in Paper - Overlap Hit Ratio
------------------------------------
Drums Generation
9
('0.61 ± 0.20', '-0.01 ± 0.05', '322.84 ± 135.25')
Groove Generation
1
('0.68 ± 0.13', '-0.03 ± 0.07', '146.79 ± 100.08')
Percent Overlapping Generated Groove
38.34 ± 22.63 %
Percent Overlapping GT Groove
48.56 ± 18.00 %
